# Manga Data Science Project

In this Project I will scrape manga data from MyAnimeList to use in a future project


#### Import Modules

In [ ]:
from requests import get 
from bs4 import BeautifulSoup
import pandas as pd
import re
import numpy as np
from time import time
from time import sleep
from random import randint
import requests
from warnings import warn

#### Planning
The code for multiple functions that will extract and clean the data from the webpage will be defined and tested .

Eventually a class with be created will these methods




#### Get request

In [ ]:
#get request of myanimelist to get html 
html = get('https://myanimelist.net/manga/1/')

####  Data Frame Structure
The data frame will use the column headings :

This data was obtained by looking at the webpage for MAL(MyAnimeList)

In [ ]:
column_head = [
  
 'English',
 'Type',
 'Volumes',
 'Chapters',
 'Status',
 'Published',
 'Genres',
 'Themes',
 'Demographic',
 'Serialization',
 'Authors',
 'Score',
 'Ranked',
 'Popularity',
 'Members',
 'Favorites']

#### Create Beautiful Soup Object

In [ ]:
soup = BeautifulSoup(html.text,'html.parser') #BS object

In [ ]:
print(soup.prettify())

#### Isolate the 'infomation' section of the page 
The information section that has the relevant statistics are the child of the class = 'spaceit_pad' 

In [ ]:
info = soup.select('.spaceit_pad  ') #information section
info = [line.text for line in info] #grab the text 
info

### Cleaning the data


#### Remove double whitespace and '\n'

In [ ]:
info = [line.replace('  ',' ').replace('\n','').strip() for line in info]
info

#### Remove columns that are not relevant and add columns that are missing

Any columns that are missing from the data will be added and have a value of 0

In [ ]:
relevant_columns = tuple(column_head) #tuple of columns needed 

info = [line for line in info if line.startswith(relevant_columns)] #grab only info that is relevant

for i in range(len(relevant_columns)): # if the info is not in the list then add it with a zero value
    col = relevant_columns[i]
    if not info[i].startswith(col):
        word = relevant_columns[i] + ': 0'
        info.insert(i,word) 
info

#### Get rid of multiple whitespaces

In [ ]:
info = [re.sub('\s+',' ',line) for line in info] #spaces removed
info = [re.sub('^\s','',line) for line in info] #start space removed 
info

#### Remove duplicates words from Genre,Themes, and Demographics

In [ ]:
#leave space after the colon to make string spliting easier later
for i in [6,7,8,9,10]:
    info[i] = info[i].replace(':',': ')
    
#Then get rid of duplicate Genre Themes and Demographic    
for i in [6,7,8]:
    info[i] = info[i].replace(',','') #Get rid of commas
    info[i] = ' '.join(dict.fromkeys(info[i].split())) 
info

To get rid of duplicate words a dictionary is created with keys equal to the words in the string that have been seperated by a white space , then the dictionary keys are joined together using whitespace

#### Clean up the score column
The score that is seen from the html.text is not the correct score, the last digit of the float must be removed 

In [ ]:
try:
    info[11] = (re.findall('.+\d+\.\d+',info[11])[0])[:-1] #remove the words from the score column
except: #If there is no score
    info[11] = 'Score: N/A'
info

#### Clean up the Ranked column

In [ ]:
#remove words from ranked column and last digit 
try:
    info[12] = (re.findall('^Ranked:\s\D\d+',info[12]))[0][:-1]  
except: #if there is no rank
    info[12] = 'Ranked: N/A'
info

#### The Date has now been cleaned
The data must now be converted into a dictionary so it can be turned into a pandas Data Frame

In [ ]:
add_row = {}  # dictionary of the row that will be added to DF

for line in info: #split keys and value with colon
    i = line.split(': ')
    add_row[i[0]] = i[1]

add_row


#### Show in Pandas Data Frame

In [ ]:
df = pd.DataFrame( columns = column_head, dtype = str) #Create Empty Data frame with columns
df_new_row = pd.DataFrame([add_row]) # row that will be added 
df = pd.concat([df, df_new_row]) # concat the data frames

df

### Automation
Now I have succesfully scraped the data of 1 webpage into the data frame , now to start the process of automating it for multiple pages

#### Monitering Request Time
If the request time is not monitered an IP ban might take place due to inhuman amounts of requests

In [ ]:
start_time =time()
request = 0

for _ in range(5):
    request += 1
    sleep(randint(1,3))
    elapsed_time = time() - start_time
    print('Request: {}; Frequency: {} requests/s'.format(request,request/elapsed_time))

#### Empty Data Frame
The Final Data Frame where the manga infomation will be stored

In [ ]:
#Final Data Frame
df = pd.DataFrame( columns = column_head, dtype = str) #Create Empty Data frame with columns 

#### URL Codes List
The URL for manga is 'https://myanimelist.net/manga/' followed by a numeric code .I will scrape the first 10,000 codes

In [ ]:
URL_NO = np.arange(7500,8500,1) #list from 1 to 5001

### Class Creation

Now I will take the previous data cleaning functions and create a class called AnimeData which will take the URL paramter and have methods to clean the data

In [ ]:
class AnimeData:
    
    
    def __init__(self,link):
        self.html = get(link)
        self.info = BeautifulSoup((self.html).text,'html.parser')
        self.column_head = [
                      'English',
                     'Type',
                     'Volumes',
                     'Chapters',
                     'Status',
                     'Published',
                     'Genres',
                     'Themes',
                     'Demographic',
                     'Serialization',
                     'Authors',
                     'Score',
                     'Ranked',
                     'Popularity',
                     'Members',
                     'Favorites']
        
        self.relevant_columns = tuple(self.column_head)
        self.EmptyDataFrame = pd.DataFrame( columns = self.column_head, dtype = str) #Create Empty Data frame with columns 
        
        
    def RelevantInfo(self):
        '''Extract Infomation statistics from the webpage'''
        
        self.info = self.info.select('.spaceit_pad  ') #information section
        self.info = [line.text for line in self.info] #grab the text 
        
        
    def RemoveNewLine(self):
        self.info = [line.replace('  ',' ').replace('\n','').strip() for line in self.info]
        
        
    def RelevantColumns(self):
        '''Check if all the relevant statistics are there
        if not add the statistic with a value of 0'''
        
        relevant_columns = tuple(self.column_head) 

        self.info = [line for line in self.info if line.startswith(relevant_columns)] #grab only info that is relevant

        for i in range(len(relevant_columns)): # if the info is not in the list then add it with a zero value
            col = str(relevant_columns[i])
            if not self.info[i].startswith(col):
                word = relevant_columns[i] + ': 0'
                self.info.insert(i,word) 


    def RemoveWhitespace(self):
        self.info = [re.sub('\s+',' ',line) for line in self.info] #spaces removed
        self.info = [re.sub('^\s','',line) for line in self.info] #start space removed 
        
        
    def RemoveDuplicateWords(self):
        for i in [6,7,8,9,10]:
            self.info[i] = self.info[i].replace(':',': ')
            

        for i in [6,7,8]:
            self.info[i] = self.info[i].replace(',','')
            self.info[i] = ' '.join(dict.fromkeys(self.info[i].split())) 
        
        
            
    def CleanScore(self):
        '''Remove last digit from score, if there is no score , Score = N/A'''
        
        try:
            self.info[11] = (re.findall('.+\d+\.\d+',info[11])[0])[:-1] #remove the words from the score column
        except:
            self.info[11] = 'Score: N/A'
        
        
            
    def CleanRank(self):
        '''Extract only the Ranked value from the string'''
        
        try:
            self.info[12] = (re.findall('^Ranked:\s\D\d+',info[12]))[0][:-1]  #remove words from ranked column and last digit 
        except:
            self.info[12] = 'Ranked: N/A'
            
        
    def AddtoDF(self,DF):
        '''Take the infomation statistics, convert it to a DataFrame 
        then concatenate the DataFrame with the FinalData frame'''
        
        add_row = {}  # dictionary of the row that will be added to DF

        for line in self.info:
            i = line.split(': ')
            add_row[i[0]] = i[1]
        
       
        df_new_row = pd.DataFrame([add_row]) # row that will be added 
        FinalDataFrame = pd.concat([DF, df_new_row]) # concat the data frames
        return FinalDataFrame
          
        

The Final Data Frame of all the data that is being collected must be defined:

In [ ]:
FinalDataFrame = pd.DataFrame( columns = column_head, dtype = str)

In [ ]:
start_time = time()
request = 0

    
for code in URL_NO:
    link = 'https://myanimelist.net/manga/'+str(code)
    manga = AnimeData(link)
    
    #limit time in between requests
    sleep(randint(8,15))
        
    #monitor the request
    request += 1
    sleep(randint(1,3))
    elapsed_time = time() - start_time
    print('Request: {}; Frequency: {} requests/s code: {}'.format(request,request/elapsed_time,code))
    
    if get(link).status_code == 200:  #OK Success Status
        manga.RelevantInfo()
        manga.RemoveNewLine()
        manga.RelevantColumns()
        manga.RemoveWhitespace()
        manga.RemoveDuplicateWords()
        manga.CleanScore()
        manga.CleanRank()
        FinalDataFrame = manga.AddtoDF(FinalDataFrame)
        
        
    else: #Webpage not found
        print(f' html status : {get(link).status_code} ; Code : {code} ; does not exist')
             

#### Save Data Frame as CSV


In [ ]:
#FinalDataFrame.to_csv('FourthAnime_Data.csv',index = False)